#Финальная работа. Часть_2 Преобразование данных расчетом экспоненциально взвешенного скользящего среднего (EWMA).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pandas pyarrow

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import psutil
import inspect
import plotly
import math
%matplotlib inline
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go


In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
pd.set_option('display.max_columns', None)
pd.options.display.float_format ='{:,.3f}'.format

#Сборка из исходных файлов

### Для преобразования признаков загружаем данные из 11 файлов 'pq' по очереди.

In [ ]:
# Функция вывод использования памяти
def get_memory_usage(dataframe):
    # Получаем имя переменной, содержащей dataframe
    dataframe_name = [name for name, obj in inspect.currentframe().f_back.f_locals.items() if obj is dataframe][0]

    # Рассчитываем использование памяти
    memory_usage = dataframe.memory_usage(deep=True).sum() / (1024 * 1024)
    memory_usage_rounded = round(memory_usage, 2)

    # Возвращаем информацию о памяти
    return f"File '{dataframe_name}': memory_usage: {memory_usage_rounded} Mb"

In [ ]:
train_11 = pd.read_parquet("/content/drive/MyDrive/Ml_Junior/Финальная работа/train_data_11.pq")


In [ ]:
train_0

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,pre_loans_outstanding,pre_loans_total_overdue,pre_loans_max_overdue_sum,pre_loans_credit_cost_rate,pre_loans5,pre_loans530,pre_loans3060,pre_loans6090,pre_loans90,is_zero_loans5,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,pre_util,pre_over2limit,pre_maxover2limit,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,enc_paym_0,enc_paym_1,enc_paym_2,enc_paym_3,enc_paym_4,enc_paym_5,enc_paym_6,enc_paym_7,enc_paym_8,enc_paym_9,enc_paym_10,enc_paym_11,enc_paym_12,enc_paym_13,enc_paym_14,enc_paym_15,enc_paym_16,enc_paym_17,enc_paym_18,enc_paym_19,enc_paym_20,enc_paym_21,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag
0,0,1,18,9,2,3,16,10,11,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
1,0,2,18,9,14,14,12,12,0,3,3,0,2,11,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,3,4,1,0,0
2,0,3,18,9,4,8,1,11,11,0,5,0,2,8,6,16,5,4,8,1,1,1,1,1,15,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,4,1,2,3,1,1,1
3,0,4,4,1,9,12,16,7,12,2,3,0,2,4,6,16,5,4,8,0,1,1,1,1,16,2,17,1,1,1,1,0,0,0,0,0,0,0,0,0,0,1,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,1,1,0,0
4,0,5,5,12,15,2,11,12,10,2,3,0,2,4,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,3,4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974719,249998,4,1,9,14,8,10,11,8,4,2,0,2,4,6,16,5,4,8,1,1,1,1,1,11,2,17,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974720,249998,5,7,9,4,8,1,11,19,1,4,0,2,7,6,16,5,4,8,1,1,1,1,1,7,2,17,0,1,1,0,0,0,0,0,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,4,1,1,1
1974721,249999,1,9,0,10,8,10,11,16,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,3,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,1
1974722,249999,2,9,16,10,13,10,4,12,2,3,0,2,7,6,16,5,4,8,1,1,1,1,1,16,2,17,1,1,1,0,0,0,0,0,0,0,0,0,3,3,4,3,3,3,3,3,3,3,3,4,3,3,3,4,1,2,3,1,0,0


# Конкатeнация данных из выбранных признаков (смотри ноутбук - Часть 2 Финальной работы)

In [ ]:
df_int8 = train_11.copy()
#Создаем список столбцов для преобразования, исключая 'id'
columns_to_convert = df_int8.columns.drop('id')

#Преобразуем выбранные столбцы в тип данных int8
df_int8[columns_to_convert] = df_int8[columns_to_convert].astype('int8')

In [ ]:
df_int8.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2284256 entries, 0 to 2284255
Data columns (total 61 columns):
 #   Column                         Dtype
---  ------                         -----
 0   id                             int64
 1   rn                             int8 
 2   pre_since_opened               int8 
 3   pre_since_confirmed            int8 
 4   pre_pterm                      int8 
 5   pre_fterm                      int8 
 6   pre_till_pclose                int8 
 7   pre_till_fclose                int8 
 8   pre_loans_credit_limit         int8 
 9   pre_loans_next_pay_summ        int8 
 10  pre_loans_outstanding          int8 
 11  pre_loans_total_overdue        int8 
 12  pre_loans_max_overdue_sum      int8 
 13  pre_loans_credit_cost_rate     int8 
 14  pre_loans5                     int8 
 15  pre_loans530                   int8 
 16  pre_loans3060                  int8 
 17  pre_loans6090                  int8 
 18  pre_loans90                    int8 
 19  

In [ ]:
get_memory_usage(train_0)

"File 'train_0': memory_usage: 919.02 Mb"

In [ ]:
get_memory_usage(df_int8)

"File 'df_int8': memory_usage: 295.86 Mb"

Как видим, один загруженный и файл занимает 919 Mb памяти, так как в файле 3 млн. строк. Да и к тому же, тип данных `int64` достаточно емкий.  Если загрузить все файлы, то будет занято много оперативной памяти, что скажется на времени вычислении (более того, ранние исследования показали, что при таких размерах файла происходит перегрузка ОЗУ). Для данных, содержащихся в столбцах датафрейма вполне подойдет тип данных `int8`. Поэтому я решила изменить тип данных на `int8`. Столбец `id` оставим в `int64`.

Видим заметную экономию ОЗУ. Теперь датафрейм с теми же данными занимает почти в 7 раз меньше памяти.

In [ ]:
del train_11

Создаем список из отобранных признаков  по важности для снижения использования оперативной памяти

In [ ]:
features_all = ['id', 'rn', 'enc_loans_credit_type', 'pre_util', 'pre_maxover2limit', 'pre_over2limit',
                'enc_paym_1', 'enc_paym_2', 'enc_paym_19', 'enc_paym_15', 'enc_paym_18', 'enc_paym_20',
                'enc_paym_21', 'enc_paym_13', 'enc_paym_23', 'enc_paym_9', 'enc_paym_4', 'enc_paym_5',
                'enc_paym_3', 'pre_loans_credit_limit', 'pre_till_pclose', 'pre_loans_next_pay_summ',
                'is_zero_util', 'pre_pterm', 'pre_loans_outstanding', 'enc_loans_credit_status', 'is_zero_loans530',
                'enc_paym_0', 'pre_loans530', 'pre_till_fclose']

In [ ]:
df_int8_new = df_int8[features_all]

In [ ]:
del df_int8

In [ ]:
feature_columns = list(df_int8_new.columns.values)
feature_columns.remove("id")
feature_columns.remove("rn")

  ## one-hot-encoding признаков

In [ ]:
dummies = pd.get_dummies(df_int8_new[feature_columns], columns=feature_columns)
dummy_features = dummies.columns.values

In [ ]:
ohe_features = pd.concat([df_int8_new, dummies], axis=1)
ohe_features = ohe_features.drop(columns=feature_columns)

Исходные данные по кредитам клиентов банка являются времянными рядами. В данных представлены кредитные истории клиентов за два года (это видно по признаку (enc_paym_{0..24} — статусы ежемесячных платежей за последние 24 месяца)

## Применение экспоненциально взвешенного скользящего среднего (EWMA) для преобразования данных в модели может быть полезным по следующим причинам:

1. Сглаживание признаков: EWMA помогает уменьшить шум в данных, сглаживая временные ряды или последовательные изменения в значениях признаков. Это может улучшить качество модели, особенно если данные содержат временные аномалии или выбросы.

2. Выявление трендов: Сглаженные значения могут лучше выявлять долгосрочные тренды и структуры в данных, которые модель может использовать для более точного прогнозирования. Это особенно важно для временных рядов и задач с временной зависимостью.

3. Уменьшение переобучения: Применение EWMA для сглаживания признаков может снизить риск переобучения модели, поскольку менее чувствительные к шуму признаки могут приводить к более обобщающим моделям.

4. Предотвращение чрезмерных колебаний: EWMA дает больше веса последним наблюдениям, что позволяет сглаживать резкие колебания данных, сохраняя при этом чувствительность к недавним изменениям. Это особенно полезно, если модель обучается на временных данных с сезонными или трендовыми компонентами.


В целом, использование EWMA в качестве одного из этапов предварительной обработки данных может повысить стабильность и качество предсказаний модели.

Расчет экспоненциально
взвешенного скользящего среднего в кредитной истории каждого клиента позволяет придать больший
вес последним значениям в кредитной истории и меньший вес — более отдаленным значениям.

Определяем длину кредитной истории клиентов (максимальное значение 'rn') c группировкой по id. Новый признак 'history_lenght'.

In [ ]:
# lenght of history
history_lenght = ohe_features.groupby('id')['rn'].max().reset_index(drop=False)
history_lenght = history_lenght.rename(columns={'rn': 'history_lenght'})
ohe_features = ohe_features.merge(history_lenght, on='id')

In [ ]:
ohe_features

,id,rn,is_zero_loans530_0,is_zero_loans530_1,pre_util_0,pre_util_1,pre_util_2,pre_util_3,pre_util_4,pre_util_5,pre_util_6,pre_util_7,pre_util_8,pre_util_9,pre_util_10,pre_util_11,pre_util_12,pre_util_13,pre_util_14,pre_util_15,pre_util_16,pre_util_17,pre_util_18,pre_util_19,is_zero_util_0,is_zero_util_1,is_zero_loans3060_0,is_zero_loans3060_1,is_zero_loans90_0,is_zero_loans90_1,is_zero_loans5_0,is_zero_loans5_1,is_zero_loans6090_0,is_zero_loans6090_1,enc_paym_1_0,enc_paym_1_1,enc_paym_1_2,enc_paym_1_3,enc_paym_0_0,enc_paym_0_1,enc_paym_0_2,enc_paym_0_3,enc_paym_2_0,enc_paym_2_1,enc_paym_2_2,enc_paym_2_3,enc_paym_3_0,enc_paym_3_1,enc_paym_3_2,enc_paym_3_3,enc_paym_4_0,enc_paym_4_1,enc_paym_4_2,enc_paym_4_3,enc_paym_5_0,enc_paym_5_1,enc_paym_5_2,enc_paym_5_3,enc_paym_18_0,enc_paym_18_1,enc_paym_18_2,enc_paym_18_3,enc_paym_7_0,enc_paym_7_1,enc_paym_7_2,enc_paym_7_3,enc_paym_6_0,enc_paym_6_1,enc_paym_6_2,enc_paym_6_3,enc_paym_13_0,enc_paym_13_1,enc_paym_13_2,enc_paym_13_3,enc_paym_16_0,enc_paym_16_1,enc_paym_16_2,enc_paym_16_3,enc_paym_19_0,enc_paym_19_1,enc_paym_19_2,enc_paym_19_3,enc_paym_9_0,enc_paym_9_1,enc_paym_9_2,enc_paym_9_3,enc_paym_20_1,enc_paym_20_2,enc_paym_20_3,enc_paym_20_4,enc_paym_17_0,enc_paym_17_1,enc_paym_17_2,enc_paym_17_3,enc_paym_14_0,enc_paym_14_1,enc_paym_14_2,enc_paym_14_3,enc_paym_10_0,enc_paym_10_1,enc_paym_10_2,enc_paym_10_3,enc_paym_21_0,enc_paym_21_1,enc_paym_21_2,enc_paym_21_3,enc_paym_15_0,enc_paym_15_1,enc_paym_15_2,enc_paym_15_3,enc_paym_22_0,enc_paym_22_1,enc_paym_22_2,enc_paym_22_3,history_lenght
0,0,1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,True,False,True,False,True,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,10
1,0,2,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,True,False,True,False,True,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,10
2,0,3,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,True,False,True,False,True,False,True,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,10
3,0,4,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,True,False,True,True,False,False,True,True,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,True,False,False,False,True,True,False,False,

Определяем вес каждого кредита в кредитной истории. И cоздаем признаки вес каждого кредита 'weight и  сумма весов 'sum_weight' с группировкой по id.

In [ ]:
ohe_features['weight'] = ohe_features['rn'] / ohe_features['history_lenght']
sum_weight = ohe_features.groupby('id')['weight'].sum().reset_index(drop=False).rename(columns={'weight': 'sum_weight'})
ohe_features = ohe_features.merge(sum_weight, on='id')

In [ ]:
generated_features = list(ohe_features.columns)
generated_features.remove("id")
generated_features.remove("rn")
generated_features.remove("history_lenght")
generated_features.remove("weight")
generated_features.remove("sum_weight")

Преобразовываем каждый признак  после one-hot-encoding умнажая на отношение веса кредита 'weight' к сумме всех весов 'sum_weight'. Эти изменения преобразовали все признаки в диапазон от 0 до 1, что поможет повысить качество модели.

In [ ]:
for feature in generated_features:
    ohe_features[feature] = ohe_features[feature] * ohe_features['weight'] / ohe_features['sum_weight']

# weight aggregation
    ohe_features.groupby("id")
    features = ohe_features.groupby("id")[generated_features].sum().reset_index(drop=False)
    features = features.merge(history_lenght, on='id')


##Так как преобразования для всех признаков после  one-hot-encoding занимают больной объем памяти даже в формате int8 мной была выбрана стратегия по отбору признаков по важности. Неполный перечень признаков конечно снизит качество модели, но других технических возможностей по преобразованиям в моем распоряжении нет.

Сохраняем преобразованные данные с отобранными признаками в 11 файлов для дольнейшего моделирования.

In [ ]:
features.to_csv('/content/drive/MyDrive/Ml_Junior/Финальная работа/Итоговая работа_все файлы/ML_сбор_итог_модель/train_11.csv', index=False)